In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#reading training data into panda dataframe
data = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')

Our NN will have a simple two-layer architecture. Input layer $a^{[0]}$ will have 784 units corresponding to the 784 pixels in each 28x28 input image. A hidden layer $a^{[1]}$ will have 10 units with ReLU activation, and finally our output layer $a^{[2]}$ will have 10 units corresponding to the ten digit classes with softmax activation.

**Forward propagation**

$$Z^{[1]} = W^{[1]} X + b^{[1]}$$
$$A^{[1]} = g_{\text{ReLU}}(Z^{[1]}))$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
$$A^{[2]} = g_{\text{softmax}}(Z^{[2]})$$

**Backward propagation**

$$dZ^{[2]} = A^{[2]} - Y$$
$$dW^{[2]} = \frac{1}{m} dZ^{[2]} A^{[1]T}$$
$$dB^{[2]} = \frac{1}{m} \Sigma {dZ^{[2]}}$$
$$dZ^{[1]} = W^{[2]T} dZ^{[2]} .* g^{[1]\prime} (z^{[1]})$$
$$dW^{[1]} = \frac{1}{m} dZ^{[1]} A^{[0]T}$$
$$dB^{[1]} = \frac{1}{m} \Sigma {dZ^{[1]}}$$

**Parameter updates**

$$W^{[2]} := W^{[2]} - \alpha dW^{[2]}$$
$$b^{[2]} := b^{[2]} - \alpha db^{[2]}$$
$$W^{[1]} := W^{[1]} - \alpha dW^{[1]}$$
$$b^{[1]} := b^{[1]} - \alpha db^{[1]}$$

**Vars and shapes**

Forward prop

- $A^{[0]} = X$: 784 x m
- $Z^{[1]} \sim A^{[1]}$: 10 x m
- $W^{[1]}$: 10 x 784 (as $W^{[1]} A^{[0]} \sim Z^{[1]}$)
- $B^{[1]}$: 10 x 1
- $Z^{[2]} \sim A^{[2]}$: 10 x m
- $W^{[1]}$: 10 x 10 (as $W^{[2]} A^{[1]} \sim Z^{[2]}$)
- $B^{[2]}$: 10 x 1

Backprop

- $dZ^{[2]}$: 10 x m ($~A^{[2]}$)
- $dW^{[2]}$: 10 x 10
- $dB^{[2]}$: 10 x 1
- $dZ^{[1]}$: 10 x m ($~A^{[1]}$)
- $dW^{[1]}$: 10 x 10
- $dB^{[1]}$: 10 x 1

In [2]:
#converting panda dataframe to numpy
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle the training set

# the data is of shape 60,000 x 785 
# 60,000 is the number of samples
# out of 785, index 0 is the output label (Y), and index 1 to 785 are the input image pixes (X)
# we need to take a transpose of "data" here so that each column represents a sample of image
data_train = data.T

#Separating label (Y) and input pixes (X)
Y_train = data_train[0]
X_train = data_train[1:n]

#Normalizing the input pixels to be within [0,1],
#this is needed otherwise Softmax layer in the Neural network overflows
X_train = X_train / 255.

In [3]:
#generating random data for Layer 1 and Layer 2
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, lr):
    W1 -= lr * dW1
    b1 -= lr * db1    
    W2 -= lr * dW2  
    b2 -= lr * db2    
    return W1, b1, W2, b2

In [6]:
def get_predictions(A2): 
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y): 
    #print(predictions, Y) 
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, lr, epochs): 
    W1, b1, W2, b2 = init_params()
    _,numsamples = X.shape
    batch = -(-numsamples//1000) #set batch = 1 for Normal Gradient Descent
    Xsplit = np.array(np.hsplit(X, batch))
    Ysplit = np.array(np.hsplit(Y, batch))

    for i in range(epochs): 
        for k in range (batch):
            Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, Xsplit[k]) 
            dW1, db1, dW2, db2 = backward_prop(Z1, A1, A2, W2, Xsplit[k], Ysplit[k]) 
            W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, lr) 
            #if i % 10 == 0: print("Iteration: ", i) 
            predictions = get_predictions(A2) 
            print(get_accuracy(predictions, Ysplit[k])) 
    return W1, b1, W2, b2

In [7]:
%time W1, b1, W2, b2 = gradient_descent(X_train, Y_train, lr=0.10, epochs=8)

0.06108333333333334
0.08423333333333333
0.09245
0.0988
0.10306666666666667
0.1067
0.11098333333333334
0.11455
CPU times: user 7.16 s, sys: 1.24 s, total: 8.41 s
Wall time: 2.3 s


Lets test the model.

In [54]:
testdata = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv')

In [55]:
testdata.shape

(10000, 785)

In [56]:
testdata = np.array(testdata)
testdata = testdata.T
rtest, ctest = testdata.shape

785 10000


In [72]:
Y_test = testdata[0]
X_test = testdata[1:rtest]
X_test = X_test / 255.

In [87]:
#Lets test a prediction
def show_image(img):
    current_image = img.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

idx=1 #the index of the image we want to choose from the test set
    
img = X_test[:,idx, None]
print(Y_test[idx]) #this will print the "digit" or "label" with this image


2


In [88]:
_,_,_,A2 = forward_prop(W1, b1, W2, b2, img)
predicted_output = np.argmax(A2,0)
print(predicted_output) #predicted output, matches the label of the test image

[2]
